In [3]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload
import io
import csv



# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive']

def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=50001)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)
    sheets = build('sheets', 'v4', credentials=creds)

    # Call the Drive v3 API
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])
    
    if not items:
        print('No files found.')
    else:
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))
    
    file_id = '1RZV-dMW4RRIE8HjvzAHgtK2kmr7wCoYAQMLroHxvjKc'
    #request = service.files().export_media(fileId=file_id,
    #                                         mimeType='application/x-vnd.oasis.opendocument.spreadsheet')
    #fh = io.FileIO('salida.csv', 'wb')
    #downloader = MediaIoBaseDownload(fh, request)
    #done = False
    #while done is False:
    #    status, done = downloader.next_chunk()
    #    print(int(status.progress() * 100))
    
    download_sheet_to_csv(sheets, file_id, 'Respuestas de formulario 1')

if __name__ == '__main__':
    main()
    
def download_sheet_to_csv(sheets_instance, spreadsheet_id, sheet_name):
    result = sheets_instance.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=sheet_name).execute()
    output_file = f'{sheet_name}.csv'

    with open(output_file, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(result.get('values'))

    f.close()

    print(f'Successfully downloaded {sheet_name}.csv')

Files:
Noticias - Vigilancia Tecnológica - Servicio de Internacionalización e Innovación (respuestas) (1RZV-dMW4RRIE8HjvzAHgtK2kmr7wCoYAQMLroHxvjKc)
Noticias - Vigilancia Tecnológica - Servicio de Internacionalización e Innovación (1H1g_4CnKnW5X_UKtBDRzXo9C2igpjLmNz3Hg13VrfRo)
20200530_Proyecto_AnalisisRRSS.pdf (13NrWIJzjkQGM0EiT_ncmYrQhl_-_y5hm)
Getting started (0B8fzYFAsjJq0c3RhcnRlcl9maWxl)
Successfully downloaded Respuestas de formulario 1.csv


# Spacy

Ahora vamos a coger alguno de ellos y vamo a intentar pasarlos por NLU's, para ver qué resultados obtenemos.

Me falta bucear en la librería para ver si puedo encontrar modelos predefinidos para el text classification. Aunque a mi lo que más me interesa es que la inteligencia artificial sea viva y que no tenga que supervisarla yo. A riesgo de depender de ella continuamente. Si no buscaré la forma de hacerme la propia.

In [1]:
import pandas as pd
df = pd.read_csv('Respuestas de formulario 1.csv')
text = df['Breve descripción de la Noticia'][401] # Escojo un texto en inglés para hacer la prueba, luego ya intentaré inferir el idioma del texto

In [11]:
# En primer lugar voy a utilizar la librería de SPACY
#!pip install spacy
#python -m spacy download en_core_web_sm
import spacy
# Load English Tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")
text
# Process whole documents
doc = nlp(text)

print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

# Find named entities, phrases and concepts

for entity in doc.ents:
    print(entity.text, entity.label_)

# Meaning Cloud
Ahora voy a utilizar MeaningCloud, una herramienta en nube que ofrece las siguientes APIs:

* Language Identification: detects the language and returns code or name
* Sentiment Analysis: detects the global polarity detected in the text
* Topics Extraction: detects the most relevant entities and concepts in the text. If the get_fibo variable is enabled, FIBO concepts will be output (requires access to the Financial Industry pack)
* Deep Categorization: categorizes the text according to the IAB 2.0 taxonomy
* Text Classification: classifies the text according the IPTC taxonomy
* Summarization: extracts a summary from the text

Para más información, acceder a su repositorio en github: 
https://github.com/MeaningCloud/meaningcloud-python

A continuación, el script de pruebas de la librería en github

In [130]:
#!pip install MeaningCloud-python
import sys
import meaningcloud
from meaningcloud_secret import API_KEY
api_key = API_KEY

# @param model str - Name of the model to use. Example: "IAB_en" by default = "IPTC_en"
model = 'IPCT_en'

# @param license_key - Your license key (found in the subscription section in https://www.meaningcloud.com/developer/)
license_key = api_key

# @param text - Text to use for different API calls
text = 'London is a very nice city but I also love Madrid.'


try:
    # We are going to make a request to the Topics Extraction API
    topics_response = meaningcloud.TopicsResponse(meaningcloud.TopicsRequest(license_key, txt=text, lang='en',
                                                                             topicType='e').sendReq())

    # If there are no errors in the request, we print the output
    if topics_response.isSuccessful():
        print("\nThe request to 'Topics Extraction' finished successfully!\n")

        entities = topics_response.getEntities()
        if entities:
            print("\tEntities detected (" + str(len(entities)) + "):\n")
            for entity in entities:
                print("\t\t" + topics_response.getTopicForm(entity) + ' --> ' +
                      topics_response.getTypeLastNode(topics_response.getOntoType(entity)) + "\n")

        else:
            print("\tNo entities detected!\n")
    else:
        if topics_response.getResponse() is None:
            print("\nOh no! The request sent did not return a Json\n")
        else:
            print("\nOh no! There was the following error: " + topics_response.getStatusMsg() + "\n")

    # CLASS API CALL
    # class_response = meaningcloud.ClassResponse(
    #   meaningcloud.ClassRequest(license_key, txt=text, model=model).sendReq())

    # SENTIMENT API CALL
    # sentiment_response = meaningcloud.SentimentResponse(
    #   meaningcloud.SentimentRequest(license_key, lang='en', txt=text, txtf='plain').sendReq())

    # GENERIC API CALL
    # generic = meaningcloud.Request(url="url_of_specific_API",key=key)
    # generic.addParam('parameter','value')
    # generic_result = generic.sendRequest()
    # generic_response = meaningcloud.Response(generic_result)

    # We are going to make a request to the Language Identification API
    lang_response = meaningcloud.LanguageResponse(meaningcloud.LanguageRequest(license_key, txt=text).sendReq())

    # If there are no errors in the request, we will use the language detected to make a request to Sentiment and Topics
    if lang_response.isSuccessful():
        print("\nThe request to 'Language Identification' finished successfully!\n")
        first_lang = lang_response.getFirstLanguage()
        if first_lang:
            language = lang_response.getLanguageCode(first_lang)
            print("\tLanguage detected: " + lang_response.getLanguageName(first_lang) + ' (' + language + ")\n")
        else:
            print("\tNo language detected!\n")

    # We are going to make a request to the Lemmatization, PoS and Parsing API
    parser_response = meaningcloud.ParserResponse(
        meaningcloud.ParserRequest(license_key, txt=text, lang='en').sendReq())

    # If there are no errors in the request, print tokenization and lemmatization
    if parser_response.isSuccessful():
        print("\nThe request to 'Lemmatization, PoS and Parsing' finished successfully!\n")
        lemmatization = parser_response.getLemmatization(True)
        print("\tLemmatization and PoS Tagging:\n")
        for token, analyses in lemmatization.items():
            print("\t\tToken -->", token)
            for analysis in analyses:
                print("\t\t\tLemma -->", analysis['lemma'])
                print("\t\t\tPoS Tag -->", analysis['pos'], "\n")


except ValueError:
    e = sys.exc_info()[0]
    print("\nException: " + str(e))


The request to 'Topics Extraction' finished successfully!

	Entities detected (2):

		London --> City

		Madrid --> City


The request to 'Language Identification' finished successfully!

	Language detected: English (en)


The request to 'Lemmatization, PoS and Parsing' finished successfully!

	Lemmatization and PoS Tagging:

		Token --> London
			Lemma --> London
			PoS Tag --> NP-S-N- 

		Token --> is
			Lemma --> be
			PoS Tag --> VI-S3PSA-N-N9 

		Token --> a
			Lemma --> a
			PoS Tag --> QD-SPN9 

		Token --> very
			Lemma --> very
			PoS Tag --> ENMSN5 

		Token --> nice
			Lemma --> nice
			PoS Tag --> AP-N6 

		Token --> city
			Lemma --> city
			PoS Tag --> NC-S-N6 

		Token --> but
			Lemma --> but
			PoS Tag --> CCAN8 

		Token --> I
			Lemma --> I
			PoS Tag --> PP-S1NN- 

		Token --> also
			Lemma --> also
			PoS Tag --> ENOPN7 

		Token --> love
			Lemma --> love
			PoS Tag --> VI-S1PSA-N-N6 

		Token --> Madrid
			Lemma --> Madrid
			PoS Tag --> NP-S-N- 

		Token --> .


Ahora vamos a utilizar uno de los ejemplos de la librería en el que se incluya una función específica para detección del idioma y otra de text Classification. Como modelo de clasificación utilizaremos, IPCT, 

In [37]:
# Calls Text Classification for a specific model and returns the label and relevance of the categories
def getTextClassification(text, model, num_cats):
    formatted_categories = ''
    print("\tGetting " + model[0:len(model) - 3].replace('_', ' ')+ " analysis...")
    class_response = meaningcloud.ClassResponse(meaningcloud.ClassRequest(license_key, txt=text, model=model, otherparams={'txtf': 'markup'}).sendReq())
    if class_response.isSuccessful():
        categories = class_response.getCategories()
        formatted_categories = (', '.join(class_response.getCategoryLabel(cat) + ' (' + class_response.getCategoryRelevance(cat) +')' for cat in categories[:num_cats])) if categories else '(none)'
    else:
        print("\tOops! The request to Text Classification was not succesful: (" + class_response.getStatusCode() + ') ' + class_response.getStatusMsg())

    return formatted_categories


# Calls Summarization and obtains an extractive summary with the number of sentences especified
def getSummarization(text, sentences):
    # We are going to make a request to the Summarization API
    summary = ''
    print("\tGetting automatic summarization...")
    summarization_response = meaningcloud.SummarizationResponse(meaningcloud.SummarizationRequest(license_key, sentences=sentences, txt=text).sendReq())
    if summarization_response.isSuccessful():
        summary = summarization_response.getSummary()
    else:
        print("\tOops! Request to Summarization was not succesful: (" + summarization_response.getStatusCode() + ') ' + summarization_response.getStatusMsg())

    return summary

def detectLanguage(text, get_name=False):
    language = ''
    # We are going to make a request to the Language Identification API
    print("\tDetecting language...")
    lang_response = meaningcloud.LanguageResponse(meaningcloud.LanguageRequest(license_key, txt=text).sendReq())
    if lang_response.isSuccessful():
        langs = lang_response.getLanguages()
        if langs:
            language = lang_response.getLanguageCode(langs[0]) if not get_name else lang_response.getLanguageName(langs[0])
    else:
        print("\tOops! Request to detect language was not succesful: (" + lang_response.getStatusCode() + ') ' + lang_response.getStatusMsg())
    return language

def extractTopics(text, fibo, relevance):
    
    entities = ''
    concepts = ''
    if fibo:
        fibo_concepts = ''

    print("\tGetting entities and concepts...") 
    topics_req = meaningcloud.TopicsRequest(license_key, txt=text, lang='en', topicType='ec', otherparams={'txtf':'markup'})
    if fibo:
        topics_req.addParam('ud', 'FIBO_en')
        
    topics_response = meaningcloud.TopicsResponse(topics_req.sendReq())

    # If there are no errors in the request, we extract the entities and concepts
    if topics_response.isSuccessful():
        entities_list = topics_response.getEntities()
        formatted_entities = []
        if entities_list:
            for entity in entities_list:
                if int(topics_response.getTopicRelevance(entity)) >= relevance: #we limit the entities to those with relevance higher than 80
                    formatted_entities.append(topics_response.getTopicForm(entity) + ' (' + topics_response.getTypeLastNode(topics_response.getOntoType(entity)) + ')')
            entities = ', '.join(formatted_entities)
        else:
            entities = '(none)'

        concepts_list = topics_response.getConcepts()
        formatted_concepts = []
        formatted_fibo_concepts = []
        if concepts_list:
            for concept in concepts_list:
                if fibo and 'dictionary' in concept.keys() and concept['dictionary'] == 'FIBO_en':
                    formatted_fibo_concepts.append(topics_response.getTopicForm(concept) + ' (' + topics_response.getTypeLastNode(topics_response.getOntoType(concept)) + ')')
                #we limit the concepts to those with relevance higher than 80 or multiwords, or user defined concepts
                elif int(topics_response.getTopicRelevance(concept)) >= relevance  or (' ' in topics_response.getTopicForm(concept) and int(topics_response.getTopicRelevance(concept)) >= (relevance/2)) or topics_response.isUserDefined(concept):
                    formatted_concepts.append(topics_response.getTopicForm(concept) + ' (' + topics_response.getTypeLastNode(topics_response.getOntoType(concept)) + ')')

            concepts = ', '.join(formatted_concepts) if formatted_concepts else '(none)'
            fibo_concepts = ', '.join(formatted_fibo_concepts) if formatted_fibo_concepts else '(none)'
        else:
            concepts = "(none)"
            fibo_concepts = "(none)"
    else:            
        print("\tOops! Request to topics was not succesful: (" + topics_response.getStatusCode() + ') ' + topics_response.getStatusMsg())


    return entities, concepts, fibo_concepts if fibo else entities, concepts

In [41]:
# @param license_key - Your license key (found in the subscription section in https://www.meaningcloud.com/developer/)
license_key = api_key
# @param get_fibo - Determines if the analysis will get FIBO concepts. Access to the pack is needed: https://www.meaningcloud.com/developer/documentation/vertical-packs#financial_industry
get_fibo = True
# @param number_categories - Number of categories to show in the results in Deep Categorization and Text Classification analysis
number_categories = 3
# @param topics_relevance - Relevance used for filtering entities and concepts
topics_relevance = 50

# @param cluster_score_threshold - Relative score used for filtering clusters
cluster_score_threshold = 50

# auxiliary variables to follow progress of the process
index_count = 1
total_files = None



In [48]:
text2 = """Con 265 positivos y 67 fallecidos por coronavirus en la última semana, la recuperación económica se ha convertido en la prioridad del Gobierno y las comunidades autónomas. Un contexto en el que el fondo de 16.000 millones no reembolsables para los territorios, que Pedro Sánchez anunció el 2 de mayo, se ha convertido en crucial. Y por tanto, ha abierto un nuevo frente político. El presidente ha explicado en la decimotercera conferencia de presidentes los nuevos criterios para el reparto del fondo, que las autonomías no tendrán que devolver. En la nueva distribución se destinarán 9.000 millones a cubrir el incremento de la inversión sanitaria, 2.000 a educación y 5.000 a la caída de ingresos. La idea original del Gobierno era que 10.000 millones fueran para compensar los gastos extraordinarios en Sanidad, otros 1.000 para gastos sociales provocados por la crisis y 5.000 más para fomentar la recuperación económica. El Congreso tendrá que aprobar el sistema de reparto. Una decisión en la que los barones del mismo partido, sean del PSOE o del PP, no se ponen de acuerdo. Los hay más interesados en que prevalezca la población y otros que piden que primen los gastos ocasionados por la covid-19 o los test realizados.


Los tramos en que se repartirá el fondo abarcarán hasta diciembre. Se empezará con 6.000 millones en julio para gastos sanitarios con criterios representativos. Otros 3.000 millones para gastos sanitarios se concederán en noviembre. Para gastos educativos se reservan 2.000 millones en septiembre: “Es uno de los hitos” ese mes, ha afirmado Sánchez. En diciembre se darían 5.000 millones enfocados a la disminución de ingresos.

Quim Torra ha sido el primero en protestar contra el cambio de criterios para la distribución del fondo no reembolsable. El president considera que les perjudica. También ha reclamado 15.000 millones de euros de inmediato “para no ahogar a Cataluña” y que el déficit pase del 0 al 1%. Como viene haciendo, ha solicitado que se libere el superávit de las entidades locales. Alberto Nuñez Feijóo también se ha mostrado contrariado por los nuevos criterios. El presidente de Galicia ha planteado que las comunidades tengan libertad para destinar la parte que les corresponda. El barón del PP ha incidido en el problema que supone la desaparición de la bolsa de 1.000 millones para gastos sociales.

Los barones del PSOE han reaccionado de manera desigual. Al presidente valenciano, Ximo Puig, le ha parecido bien los nuevos criterios. Emiliano García-Page, en cambio, ha pedido que se reflexione el gasto vinculado a educación. A otro presidente socialista, el aragonés Javier Lambán, el fondo de 2.000 millones para educación le parece bien pero ha apuntado que habría que tener en cuenta el coste en el mundo rural, donde la dispersión geográfica hace que se triplique o sea cuatro veces superior.

Tras la polémica de esta semana, en la que el Gobierno rectificó el anuncio de que abriría las fronteras el 21 de junio, el presidente ha trasladado, según fuentes de la reunión, que su reapertura será el 1 de julio. Sánchez también aboga por una propuesta conjunta con Italia para que haya normas comunes bajo criterios epidemiológicos tanto dentro como fuera de la zona Schengen. Feijóo ha reclamado la apertura de más pasos fronterizos entre Galicia y el norte de Portugal.

La cumbre precede a la entrada este lunes de 13 comunidades en la fase 3, en la que la desescalada dependerá de los Ejecutivos autonómicos salvo la restricción de la movilidad. “La filosofía es la que os planteé el domingo pasado. Pasamos de una cogobernanza a una gobernanza total de las comunidades en la fase 3 en ese tránsito a la nueva normalidad”, ha expuesto Sánchez. Iñigo Urkullu ha pedido que se permita la movilidad con Cantabria, Navarra y La Rioja a partir del 15 de junio. Esto es, una semana antes del final del estado de alarma. Sánchez ha subrayado en la cumbre, según distintas fuentes presentes, que las comunidades en fase 3 podrán solicitar cuando quieran levantar el estado de alarma antes del 21 de junio y acordar la movilidad con otras que estén en la misma situación. El presidente cántabro, Miguel Ángel Revilla, ha coincidido con la reivindicación de Urkullu de permitir la movilidad con los territorios limítrofes en la fase 3.

"""

In [49]:
# Language detection
lenguaje = detectLanguage(text2)
print(lenguaje)


	Detecting language...
es


In [50]:
#Text Summarization
textoresumido = getSummarization(text2,2)
print(textoresumido)

	Getting automatic summarization...
Con 265 positivos y 67 fallecidos por coronavirus en la última semana, la recuperación económica se ha convertido en la prioridad del Gobierno y las comunidades autónomas. El presidente ha explicado en la decimotercera conferencia de presidentes los nuevos criterios para el reparto del fondo, que las autonomías no tendrán que devolver.


In [118]:
# Text Classification
getTextClassification(textoresumido,'IPTC_es', 8)

	Getting IPTC analysis...


'política - gobierno (100), política - gobierno - ministros (gobierno) (89), economía, negocios y finanzas - economía (general) (70)'

In [58]:
# Extract topics
text3 = """TThe Queen chose an intricate diamond and turquoise brooch for her most recent public address during the coronavirus pandemic. The piece was gifted to her grandmother, Queen Mary, on the day of her wedding to King George V in 1893, and was inherited by Elizabeth at the time of her coronation. The Queen has rarely been seen wearing it, and some have speculated that she chose to wear it for her landmark televised speech due its longstanding history in the collections of monarchs who have faced national difficulties – Queen Mary, for example, supported her husband, King George V, throughout the First World War.  

Below, a brief history of a selection of Queen Elizabeth II’s storied brooches."""
fibo = False
topics = extractTopics(text2, fibo, topics_relevance)

	Getting entities and concepts...


In [59]:
topics

('El Mundo (Newspaper), presidente de Galicia (Title), Pedro Sánchez (FullName), Fase (LastName), Una (River)',
 'coronavirus (Disease)',
 'El Mundo (Newspaper), presidente de Galicia (Title), Pedro Sánchez (FullName), Fase (LastName), Una (River)',
 'coronavirus (Disease)')

Ahora vamos a intentar coger cada una de nuestras noticias resumidas y pasarle los filtros que indicamos para ver si es capaz de hacerlo correctamente. Para ello vamos a seguir los siguientes pasos:
1. Leer cada línea de noticias
2. Detectar el lenguaje en el que están escritos
3. (Cuando tenga tiempo, veré como puedo hacer un webscrapping del texto completo de cada uno de las noticias para hacer el Text Summarization) - Me pondré en primer lugar a ver como sacar el texto de las convocatorias europeas para ver si se puede hacer más eficaz la 
4. Haremos el TextClassification
5. Extraeremos los topics
6. Guardar todo el Dataframe
6. Lo guardaremos todo el una BBDD MongoDB para su posterior consulta y ya poder trabajar con ello. Para la introducción del dataframe en MongoDB tomar como ejemplo el siguiente bloque de código:


         from pandas_datareader import data as pdr
         import pandas as pd
         from pymongo import MongoClient
         import yfinance as yf
         yf.pdr_override()
 


We get data from yahoo finance 

 ### download dataframe
         data1 = pdr.get_data_yahoo("SPY", start="2017-01-01", end="2017-01-15"

Now, we make a connection to Mongodb.

Here I am connecting to local MongoDb Server on port 27018 and then creating a database with the name `finance`. MongoDB has a collection(table) and I name it as `mycollection`.

         #Step 1: Connect to MongoDB - Note: Change connection string as needed
         myclient = MongoClient("mongodb://localhost:27017/")
         mydb = myclient["finance"]
         mycol = mydb["mycollection"]

Now that you have the data (data1), you can insert into the MongoDB database. But before it, you have to do convert the data frame into a dictionary. The other thing is that the Date column is set as Index of the Dataframe, therefore you have to reset the index before inserting.

 ### Step 2: Insert Data into DB
         data1.reset_index(inplace=True) # Reset Index
         data_dict = data1.to_dict("records") # Convert to dictionary
         mycol.insert_one({"index":"SPY","data":data_dict}) # inesrt into D


In [106]:
# Detect language
def detectLanguagedf(row):
    text = row['Breve descripción de la Noticia']
    get_name=False
    language = ''
    # We are going to make a request to the Language Identification API
    print("\tDetecting language...")
    lang_response = meaningcloud.LanguageResponse(meaningcloud.LanguageRequest(license_key, txt=text).sendReq())
    if lang_response.isSuccessful():
        langs = lang_response.getLanguages()
        if langs:
            language = lang_response.getLanguageCode(langs[0]) if not get_name else lang_response.getLanguageName(langs[0])
    else:
        print("\tOops! Request to detect language was not succesful: (" + lang_response.getStatusCode() + ') ' + lang_response.getStatusMsg())
    return language



In [107]:
df['language'] = df.apply(detectLanguagedf, axis=1)

	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting 

	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting language...
	Detecting 

In [114]:
df['Breve descripción de la Noticia'][df.language == '']

97    https://www.puromarketing.com/13/33706/puntos-...
Name: Breve descripción de la Noticia, dtype: object

In [119]:
# Text Classification
def getTextClassificationdf(row):
    text = row['Breve descripción de la Noticia']
    if row.language == 'en':
        model = 'IPTC_en'
    else:
        model = 'IPTC_es'
    num_cats = 8
    formatted_categories = ''
    print("\tGetting " + model[0:len(model) - 3].replace('_', ' ')+ " analysis...")
    class_response = meaningcloud.ClassResponse(meaningcloud.ClassRequest(license_key, txt=text, model=model, otherparams={'txtf': 'markup'}).sendReq())
    if class_response.isSuccessful():
        categories = class_response.getCategories()
        formatted_categories = (', '.join(class_response.getCategoryLabel(cat) + ' (' + class_response.getCategoryRelevance(cat) +')' for cat in categories[:num_cats])) if categories else '(none)'
    else:
        print("\tOops! The request to Text Classification was not succesful: (" + class_response.getStatusCode() + ') ' + class_response.getStatusMsg())
    
    print(formatted_categories)
    return formatted_categories


In [120]:
df['text_classification'] = df.apply(getTextClassificationdf, axis=1)

	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
economía, negocios y finanzas - información para empresas - investigación y desarrollo (100)
	Getting IPTC analysis...
medio ambiente - energía alternativa (100), medio ambiente - política ambiental (79)
	Getting IPTC analysis...
economía, negocios y finanzas - economía (general) (100), economía, negocios y finanzas (94), economía, negocios y finanzas - macroeconomía - empresas (63)
	Getting IPTC analysis...
economía, negocios y finanzas - información para empresas - investigación y desarrollo (100), ciencia y tecnología - investigación (90)
	Getting IPTC analysis...
economía, negocios y finanzas - macroeconomía - empresas (100), economía, negocios y finanzas - economía (general) (68)
	Getting IPTC analysis...
economía, negocios y finanzas - agricultura - acuacultura (100)
	Getting IPTC analysis...
meteorología - boletines (100), ciencia y tecnología - ciencias (general) (98)
	Getting IPTC analysis...
(none)
	Getting IPTC analy

(none)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
salud - epidemias y plagas (100), economía, negocios y finanzas - economía (general) (78)
	Getting IPTC analysis...
mano de obra - empleados (100)
	Getting IPTC analysis...
economía, negocios y finanzas - macroeconomía - préstamos (100)
	Getting IPTC analysis...
economía, negocios y finanzas - economía (general) (100)
	Getting IPTC analysis...
educación (100), economía, negocios y finanzas - economía (general) (84), salud (67), economía, negocios y finanzas - turismo y tiempo libre (52)
	Getting IPTC analysis...
ciencia y tecnología - investigación (100)
	Getting IPTC analysis...
medio ambiente - energía alternativa (100), educación - universidad (82)
	Getting IPTC analysis...
meteorología - cambios climáticos (100), medio ambiente - energía alternativa (50)
	Getting IPTC analysis...
ciencia y tecnología - investigación (100), ciencia y tecnología - ciencias aplicadas (77), ciencia y tecnología - ciencias (general) (51)


education - university (100), economy, business and finance - computing and information technology - hardware (60)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
política - defensa (100), policía y justicia - policía (82), arte, cultura y espectáculos - medios masivos (66)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
economía, negocios y finanzas - economía (general) (100)
	Getting IPTC analysis...
salud - medicina preventiva - vacunas (100)
	Getting IPTC analysis...
economía, negocios y finanzas - computación e informática - redes (100)
	Getting IPTC analysis...
ciencia y tecnología - tecnología (general) (100)
	Getting IPTC analysis...
ciencia y tecnología - ciencias aplicadas (100), ciencia y tecnología - investigación (94)
	Getting IPTC analysis...
ciencia y tecnología - investigación (100), salud - enfermedades (82), salud - enfermedades - enfermedades víricas (75)
	Getting IPTC analysis...
arte, cultura y espectáculos - lenguaje (100)
	Getting IPTC analy

ciencia y tecnología - ciencias (general) (100)
	Getting IPTC analysis...
arte, cultura y espectáculos - internet (100)
	Getting IPTC analysis...
economía, negocios y finanzas - economía (general) (100), economía, negocios y finanzas - medios de difusión - publicidad (51)
	Getting IPTC analysis...
economía, negocios y finanzas (100), economía, negocios y finanzas - economía (general) (71)
	Getting IPTC analysis...
science and technology - engineering (100), science and technology - applied science - particle physics (93), arts, culture and entertainment - fashion (55), economy, business and finance - computing and information technology - hardware (52)
	Getting IPTC analysis...
ciencia y tecnología - ciencias (general) (100)
	Getting IPTC analysis...
ciencia y tecnología (100)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
economía, negocios y finanzas - macroeconomía - presupuestos (100), economía, negocios 

science and technology - natural science - astronomy (100)
	Getting IPTC analysis...
mano de obra - empleo (100), economía, negocios y finanzas - macroeconomía - inversiones (83), mano de obra (64), arte, cultura y espectáculos - cultura (general) (59), asuntos sociales (53)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
educación - universidad (100)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
política - políticas internas - protección de datos (100), economía, negocios y finanzas - computación e informática - seguridad (72)
	Getting IPTC analysis...
economy, business and finance - computing and information technology - telecommunication equipment (100), economy, business and finance - computing and information technology - wireless technology (86)
	Getting IPTC analysis...
ciencia y tecnología (100)
	Getting IPTC analysis...
economía, negocios y finanzas - productos químicos - productos farmacéuticos (100), salud - medicamentos (93)
	Getting IPTC analysis...

(none)
	Getting IPTC analysis...
science and technology - technology (general) (100)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
economía, negocios y finanzas - turismo y tiempo libre (100)
	Getting IPTC analysis...
educación - escuelas (100)
	Getting IPTC analysis...
ciencia y tecnología - ciencias (general) (100), ciencia y tecnología (93), política - elecciones (53)
	Getting IPTC analysis...
economía, negocios y finanzas (100), economía, negocios y finanzas - macroeconomía - empresas (94), asuntos sociales (61)
	Getting IPTC analysis...
economía, negocios y finanzas - economía (general) (100)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
arte, cultura y espectáculos - internet (100)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
weather - forecast (100)
	Getting IPTC analysis...
economía, negocios y finanzas - macroeconomía - crédito y débito (100)
	Getting IPTC analysis...
(none)
	Getting IPTC analysis...
política - gobierno (100)
	Getting IP

In [122]:
# Extracción de los topics
def extractTopicsdf(row):
    fibo = False
    text = row['Breve descripción de la Noticia']
    relevance = 75
    entities = ''
    concepts = ''
    if fibo:
        fibo_concepts = ''

    print("\tGetting entities and concepts...") 
    topics_req = meaningcloud.TopicsRequest(license_key, txt=text, lang='en', topicType='ec', otherparams={'txtf':'markup'})
    if fibo:
        topics_req.addParam('ud', 'FIBO_en')
        
    topics_response = meaningcloud.TopicsResponse(topics_req.sendReq())

    # If there are no errors in the request, we extract the entities and concepts
    if topics_response.isSuccessful():
        entities_list = topics_response.getEntities()
        formatted_entities = []
        if entities_list:
            for entity in entities_list:
                if int(topics_response.getTopicRelevance(entity)) >= relevance: #we limit the entities to those with relevance higher than 80
                    formatted_entities.append(topics_response.getTopicForm(entity) + ' (' + topics_response.getTypeLastNode(topics_response.getOntoType(entity)) + ')')
            entities = ', '.join(formatted_entities)
        else:
            entities = '(none)'

        concepts_list = topics_response.getConcepts()
        formatted_concepts = []
        formatted_fibo_concepts = []
        if concepts_list:
            for concept in concepts_list:
                if fibo and 'dictionary' in concept.keys() and concept['dictionary'] == 'FIBO_en':
                    formatted_fibo_concepts.append(topics_response.getTopicForm(concept) + ' (' + topics_response.getTypeLastNode(topics_response.getOntoType(concept)) + ')')
                #we limit the concepts to those with relevance higher than 80 or multiwords, or user defined concepts
                elif int(topics_response.getTopicRelevance(concept)) >= relevance  or (' ' in topics_response.getTopicForm(concept) and int(topics_response.getTopicRelevance(concept)) >= (relevance/2)) or topics_response.isUserDefined(concept):
                    formatted_concepts.append(topics_response.getTopicForm(concept) + ' (' + topics_response.getTypeLastNode(topics_response.getOntoType(concept)) + ')')

            concepts = ', '.join(formatted_concepts) if formatted_concepts else '(none)'
            fibo_concepts = ', '.join(formatted_fibo_concepts) if formatted_fibo_concepts else '(none)'
        else:
            concepts = "(none)"
            fibo_concepts = "(none)"
    else:            
        print("\tOops! Request to topics was not succesful: (" + topics_response.getStatusCode() + ') ' + topics_response.getStatusMsg())


    return entities, concepts, fibo_concepts if fibo else entities, concepts

In [123]:
df['topics']=df.apply(extractTopicsdf, axis=1)

	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entit

	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entities and concepts...
	Getting entit

In [124]:
df

,Marca temporal,Titulo de la Noticia,Día de Publicación,Fuente,"Si contestó 'otra', ¿De qué fuente proviene?",Breve descripción de la Noticia,Incrustar etiquetas,"Si no encuentra la etiqueta adecuada, ¿Qué etiqueta añadiría a su noticia? (Separe los tags por comas)",Link de la noticia,¿Tiene lugar en La Rioja?,language,text_classification,topics
0,5/06/2020 9:01:12,Guía para la evaluación nacional de competenci...,5/06/2020,smartcity.es,NaN,La guía publicada por la ITU ayudará a los Est...,"Innovación, Telecomunicaciones, Digitalización...",NaN,https://www.esmartcity.es/2020/06/05/guia-eval...,No,es,(none),"(Guia (LastName), International Telecommunicat..."
1,5/06/2020 9:15:40,"Resolución de 2 de junio de 2020, de la Agenc...",5/06/2020,BOR,NaN,"Resolución de 2 de junio de 2020, de la Agenc...","Innovación, Ayudas, Investigación, Subvenciones",NaN,http://ias1.larioja.org/boletin/Bor_Boletin_vi...,Sí,es,"economía, negocios y finanzas - información pa...","(Desarrollo (Top), (none), Desarrollo (Top), (..."
2,5/06/2020 9:18:25,"Resolución de 2 de junio de 2020, de la Agenc...",5/06/2020,BOR,NaN,"Resolución de 2 de junio de 2020, de la Agenc...","Energía, Economía, Reciclaje, Medioambiente",NaN,http://ias1.larioja.org/boletin/Bor_Boletin_vi...,Sí,es,"medio ambiente - energía alternativa (100), me...",(Agencia de Desarrollo Económico de La Rioja (...
3,5/06/2020 9:21:20,"Resolución de 2 de junio de 2020, de la Agenc...",5/06/2020,BOR,NaN,"Resolución de 2 de junio de 2020, de la Agenc...","Ayudas, Subvenciones, Empresa",NaN,https://web.larioja.org/bor-portada/boranuncio...,Sí,es,"economía, negocios y finanzas - economía (gene...",(Agencia de Desarrollo Económico de La Rioja (...
4,5/06/2020 9:21:21,"Resolución de 2 de junio de 2020, de la Agenci...",5/06/2020,BOR,NaN,Subvenciones destinadas al fomento de la inves...,"Convocatorias Nacionales, Proyectos",NaN,https://web.larioja.org/bor-portada/boranuncio...,Sí,es,"economía, negocios y finanzas - información pa...","(Subvenciones (Top), Al (FirstName), (none), S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,26/05/2020 13:05:32,"WunderApp, reconocido como proyecto emprendedo...",2/01/2020,FER,NaN,Manuel Fernández de Tejada y Juan Ramón Gonzál...,Emprendedor,NaN,https://sie.fer.es/esp/Servicios/Comunicacion/...,Sí,es,(none),"(Manuel Fernández de Tejada (Top), Juan Ramón ..."
398,26/05/2020 13:18:43,R.I.P. Windows 7 y Server 2018,2/01/2020,INCIBE (Centro Nacional de Ciberseguridad),NaN,Microsoft va a dejar de dar soporte técnico a ...,"Ciberseguridad, Ingeniería de Software, Sector...",NaN,https://www.incibe.es/protege-tu-empresa/blog/...,No,es,"economía, negocios y finanzas - computación e ...","(Microsoft Corporation (SoftwareCompany), Una ..."
399,26/05/2020 13:21:49,"El CDTI concede 106,03 millones de euros a 166...",2/01/2020,CDTI,NaN,El Consejo de Administración del Centro para e...,"Innovación, Convocatorias Nacionales, Ayudas, ...",NaN,http://www.cdti.es/index.asp?MP=9&MS=31&MN=2&T...,No,es,"economía, negocios y finanzas - información pa...",(El Consejo de Administración del Centro para ...
400,26/05/2020 13:16:36,El capital extranjero dispara el valor de las ...,1/01/2020,Expansión,NaN,El interés de los inversores foráneos por el s...,"Inversión, startup",NaN,https://www.expansion.com/empresas/2020/01/01/...,No,es,(none),"(El Soplao (LandForm), como Glovo (Top), segun..."


In [125]:
# Ahora vamos a meter el dataframe en MongoDB
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client['BoletinNoticias']
collection_region = db['AnalisisNoticias']
df.reset_index(inplace=True) 
bbdd_dict = df.to_dict("records") # Convert to dictionary
x = collection_region.insert_many(bbdd_dict)

